## Task 3: Building a Database of Summary Tables 

In [5]:
# Import dependencies 

from google.cloud import bigquery
import pandas as pd
import sqlite3

In [2]:
# Define Credentials & Establish Connection

gbq_proj_id = "dow-wedge-transactions"
dataset_id = "transactions"  

client = bigquery.Client(project=gbq_proj_id)

In [6]:
# Establish SQLite connection, creating Wedge Summary database (.db) file

conn = sqlite3.connect('wedge_summary.db')
cursor = conn.cursor()

In [ ]:
# Sales by Date and Hour (GBQ query and table insertion to SQL database)

# Query to get sales by date by hour
sales_by_date_hour_query = """
SELECT
  EXTRACT(DATE FROM datetime) AS sale_date,
  EXTRACT(HOUR FROM datetime) AS sale_hour,
  SUM(total) AS total_spend,
  COUNT(DISTINCT trans_no) AS total_transactions,
  COUNT(CASE WHEN trans_status = '' OR trans_status = ' ' THEN 1 END) AS total_items
FROM `dow-wedge-transactions.transactions.transArchive_*`
WHERE card_no != 3
  AND trans_status NOT IN ('R', 'V')  -- Exclude returns and voids
GROUP BY sale_date, sale_hour
ORDER BY sale_date, sale_hour;
"""

# Run the query and get results
sales_by_date_hour_results = client.query(sales_by_date_hour_query).result() # saved in memory

cursor.execute('DROP TABLE IF EXISTS sales_by_date_by_hour') # deleting table if exists, as I updated the query 

# Create the table for sales by date by hour
cursor.execute('''
CREATE TABLE IF NOT EXISTS sales_by_date_by_hour (
    sale_date TEXT,
    sale_hour INTEGER,
    total_spend REAL,
    total_transactions INTEGER,
    total_items INTEGER
)
''')

# Insert the data into the SQLite table
for row in sales_by_date_hour_results:
    cursor.execute('''
    INSERT INTO sales_by_date_by_hour (sale_date, sale_hour, total_spend, total_transactions, total_items)
    VALUES (?, ?, ?, ?, ?)
    ''', (row.sale_date, row.sale_hour, row.total_spend, row.total_transactions, row.total_items))

# Commit the transaction and close the connection (standard practice)
conn.commit()
conn.close()

print("Sales by Date by Hour table created and populated.")

In [ ]:
# Sales by Owner by Year and Month (GBQ query and table insertion to SQL database)

conn = sqlite3.connect('wedge_summary.db')
cursor = conn.cursor()

# Query to get sales by owner by year by month
sales_by_owner_query = """
SELECT
  card_no,
  EXTRACT(YEAR FROM datetime) AS year,
  EXTRACT(MONTH FROM datetime) AS month,
  SUM(total) AS total_spend,
  COUNT(DISTINCT trans_no) AS total_transactions,
  COUNT(CASE WHEN trans_status = '' OR trans_status = ' ' THEN 1 END) AS total_items
FROM `dow-wedge-transactions.transactions.transArchive_*`
WHERE card_no != 3
GROUP BY card_no, year, month
ORDER BY card_no, year, month;
"""

# Run the query and get results
sale_by_owner_results = client.query(sales_by_owner_query).result() 

# Deleting table if exists
cursor.execute('DROP TABLE IF EXISTS sales_by_owner_by_year_by_month') 

# Create the table for sales by owner by year by month
cursor.execute('''
CREATE TABLE IF NOT EXISTS sales_by_owner_by_year_by_month (
    card_no TEXT,
    year INTEGER,
    month INTEGER,
    total_spend REAL,
    total_transactions INTEGER,
    total_items INTEGER
)
''')

# Insert the data into the SQLite table
for row in sale_by_owner_results:
    cursor.execute('''
    INSERT INTO sales_by_owner_by_year_by_month (card_no, year, month, total_spend, total_transactions, total_items)
    VALUES (?, ?, ?, ?, ?, ?)
    ''', (row.card_no, row.year, row.month, row.total_spend, row.total_transactions, row.total_items))

# Commit the transaction and close the connection
conn.commit()
conn.close()

print("Sales by Owner by Year by Month table created and populated.")

In [ ]:
# Create the Department Lookup table, upload to GBQ (Lookup info from Wedge Project description)

# Copy over department lookup data via List
departments = [
    (1, 'PACKAGED GROCERY'),
    (2, 'PRODUCE'),
    (3, 'BULK'),
    (4, 'REF GROCERY'),
    (5, 'CHEESE'),
    (6, 'FROZEN'),
    (7, 'BREAD'),
    (8, 'DELI'),
    (9, 'GEN MERCH'),
    (10, 'SUPPLEMENTS'),
    (11, 'PERSONAL CARE'),
    (12, 'HERBS&SPICES'),
    (13, 'MEAT'),
    (14, 'JUICE BAR'),
    (15, 'MISC P/I'),
    (16, 'FISH&SEAFOOD'),
    (17, 'BAKEHOUSE'),
    (18, 'FLOWERS'),
    (19, 'WEDGEWORLDWIDE'),
    (20, 'MISC P/I - WWW'),
    (21, 'CATERING'),
    (22, 'BEER & WINE')
]

# Convert to a pandas DataFrame
df_departments = pd.DataFrame(departments, columns=['department_number', 'department_name'])

# Upload the department_lookup table to BigQuery
table_id = f"{gbq_proj_id}.{dataset_id}.department_lookup"
job = client.load_table_from_dataframe(df_departments, table_id)

# Wait for the job to complete
job.result()

print(f"Department lookup table uploaded to BigQuery: {table_id}")

In [ ]:
# Sales by Product Description by Year and Month (GBQ query and table insertion to SQL database)

conn = sqlite3.connect('wedge_summary.db')
cursor = conn.cursor()

# Query to get sales by product description by year by month (matching department name to number)
sales_by_product_query = """
WITH sales_by_product_by_year_by_month AS (
  SELECT
    upc,
    description,
    department,
    EXTRACT(YEAR FROM datetime) AS year,
    EXTRACT(MONTH FROM datetime) AS month,
    SUM(total) AS total_spend,
    COUNT(DISTINCT trans_no) AS total_transactions,
    COUNT(CASE WHEN trans_status = '' OR trans_status = ' ' THEN 1 END) AS total_items
  FROM `dow-wedge-transactions.transactions.transArchive_*`
  WHERE card_no != 3
  GROUP BY upc, description, department, year, month
)

SELECT
  sp.upc,
  sp.description,
  sp.department,
  dl.department_name,
  sp.year,
  sp.month,
  sp.total_spend,
  sp.total_transactions,
  sp.total_items
FROM sales_by_product_by_year_by_month sp
LEFT JOIN `dow-wedge-transactions.transactions.department_lookup` dl  -- Reference to BigQuery table
ON sp.department = dl.department_number
ORDER BY sp.upc, sp.year, sp.month;
"""

# Run the query and get results
sales_by_product_results = client.query(sales_by_product_query).result()

# Deleting table if exists, as I updated the query 
cursor.execute('DROP TABLE IF EXISTS sales_by_product_by_year_by_month') 

# Create the table for sales by product description by year by month
cursor.execute('''
CREATE TABLE IF NOT EXISTS sales_by_product_by_year_by_month (
    upc TEXT,
    description TEXT,
    department INTEGER,
    department_name TEXT,
    year INTEGER,
    month INTEGER,
    total_spend REAL,
    total_transactions INTEGER,
    total_items INTEGER
)
''')

# Insert the data into the SQLite table
for row in sales_by_product_results:
    cursor.execute('''
    INSERT INTO sales_by_product_by_year_by_month (upc, description, department, department_name, year, month, total_spend, total_transactions, total_items)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (row.upc, row.description, row.department, row.department_name, row.year, row.month, row.total_spend, row.total_transactions, row.total_items))

# Commit the transaction and close the connection
conn.commit()
conn.close()

print("Sales by Product Description by Year by Month table created and populated.")